In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
standings_url = "https://fbref.com/en/comps/11/Serie-A-Stats"
data = requests.get(standings_url)
data

<Response [200]>

In [3]:
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
data = requests.get(team_urls[0])

In [4]:
years = [3,2,1]
years

[3, 2, 1]

In [5]:
all_matches= []

In [6]:
standings_url = "https://fbref.com/en/comps/11/Serie-A-Stats"

In [7]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/keeper' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        keeping = pd.read_html(data.text, match="Goalkeeping")[0]
        keeping.columns = keeping.columns.droplevel()
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/passing' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        passing = pd.read_html(data.text, match="Passing")[0]
        passing= passing[[f'For {team_name}','Total']]
        passing.columns= passing.columns.droplevel()
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/misc/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        mish = pd.read_html(data.text, match="Miscellaneous Stats ")[0]
        mish.columns = mish.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "SoT%", "G/SoT", "Dist", "G-xG"]], on="Date")
        except ValueError:
            continue
        try:
            team_data = team_data.merge(keeping[["Date","Save%", "Stp%"]], on="Date")
        except ValueError:
            continue
        try:
            team_data = team_data.merge(passing[["Date", "Cmp%", "TotDist"]], on="Date")
        except ValueError:
            continue
        try:
            team_data = team_data.merge(mish[["Date",  "CrdY", "CrdR", "Fls","Recov"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Serie A"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [9]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,save%,stp%,cmp%,totdist,crdy,crdr,fls,recov,season,team
0,2023-08-19,20:45,Serie A,Matchweek 1,Sat,Home,W,2.0,0.0,Monza,...,100.0,10.0,88.0,7722.0,1,0,8.0,40.0,3,Internazionale
1,2023-08-28,20:45,Serie A,Matchweek 2,Mon,Away,W,2.0,0.0,Cagliari,...,100.0,0.0,84.4,10857.0,1,0,10.0,63.0,3,Internazionale
2,2023-09-03,18:30,Serie A,Matchweek 3,Sun,Home,W,4.0,0.0,Fiorentina,...,100.0,0.0,81.7,5579.0,1,0,13.0,53.0,3,Internazionale
3,2023-09-16,18:00,Serie A,Matchweek 4,Sat,Home,W,5.0,1.0,Milan,...,50.0,16.7,80.9,5601.0,2,0,16.0,31.0,3,Internazionale
5,2023-09-24,12:30,Serie A,Matchweek 5,Sun,Away,W,1.0,0.0,Empoli,...,100.0,11.1,83.3,8961.0,2,0,11.0,49.0,3,Internazionale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2022-05-01,12:30,Serie A,Matchweek 35,Sun,Away,L,1,2,Juventus,...,50.0,0.0,80.8,6492.0,3,0,12.0,49.0,1,Venezia
37,2022-05-05,18:00,Serie A,Matchweek 20,Thu,Away,L,1,2,Salernitana,...,75.0,0.0,74.9,6225.0,7,1,17.0,38.0,1,Venezia
38,2022-05-08,15:00,Serie A,Matchweek 36,Sun,Home,W,4,3,Bologna,...,70.0,0.0,74.8,4786.0,2,0,10.0,52.0,1,Venezia
39,2022-05-14,20:45,Serie A,Matchweek 37,Sat,Away,D,1,1,Roma,...,93.8,2.6,69.3,3398.0,5,1,14.0,40.0,1,Venezia


In [10]:
match_df.to_csv(r"C:\Users\nicol\Desktop\git\matches_2021_24.csv")